# Data Cleaning Listado de Listado de Beneficiarios 2020

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_20 -> Dataset con el listado de TODOS los beneficiarios de 2020.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2020

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_20 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2020.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_20 = beneficiarios_20[~((beneficiarios_20['ENTIDAD'] == 'NACIONAL') & (beneficiarios_20['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_20['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_20.shape

(369040, 16)

In [6]:
beneficiarios_20['ENTIDAD'].unique()

array(['GUERRERO', 'MORELOS', 'TLAXCALA', 'PUEBLA', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [7]:
beneficiarios_20.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Localidades_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_20 = Localidades_20.drop_duplicates()


In [9]:
Localidades_20['ENTIDAD_c_benef'] = Localidades_20['ENTIDAD'].apply(clean_text)
Localidades_20['MUNICIPIO_c_benef'] = Localidades_20['MUNICIPIO'].apply(clean_text)
Localidades_20['LOCALIDAD_c_benef'] = Localidades_20['LOCALIDAD'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 2019
Localidades_20['KEY_benef_mun'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_20['KEY_benef_loc'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_20['LOCALIDAD_c_benef'].astype(str)

In [11]:
Localidades_20.shape

(5381, 8)

In [12]:
# Obtenemos las localidades únicas en el dataset.
Municipios_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO']]
Municipios_20 = Municipios_20.drop_duplicates()

In [13]:
# Estandarizamos la limpieza de los datos
Municipios_20['ENTIDAD_c_benef'] = Municipios_20['ENTIDAD'].apply(clean_text)
Municipios_20['MUNICIPIO_c_benef'] = Municipios_20['MUNICIPIO'].apply(clean_text)

In [14]:
# Creamos las dos keys de beneficiarios 
Municipios_20['KEY_benef_mun'] = Municipios_20['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_20[
    'MUNICIPIO_c_benef'].astype(str)

In [15]:
Municipios_20.shape

(182, 5)

In [16]:
Municipios_20.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2020

Obtener listado único de municipios y listado único de localidades de inegi de 2020 con sus claves.

In [17]:
path_dataset_inegi_2020 = '../../data/inegi/dataset_inegi_clean_2020.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2020 = pd.read_csv(path_dataset_inegi_2020)

In [ ]:
dataset_inegi_2020['CVE_ENT'] = dataset_inegi_2020['CVE_ENT'].astype(str).str.zfill(2)
dataset_inegi_2020['CVE_MUN'] = dataset_inegi_2020['CVE_MUN'].astype(str).str.zfill(3)
dataset_inegi_2020['CVE_LOC'] = dataset_inegi_2020['CVE_LOC'].astype(str).str.zfill(4)

### 1.2.1 INEGI 2020 Municipios únicos para cada año.

In [18]:
dataset_inegi_2020['KEY_inegi_municipio'] = dataset_inegi_2020['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2020[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2020['KEY_inegi_localidad'] = dataset_inegi_2020['Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2020['Localidad_c_inegi'].astype(str)

In [19]:
INEGI_UNIQUEMUN_2020 = dataset_inegi_2020.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])
INEGI_UNIQUEMUN_2020 = INEGI_UNIQUEMUN_2020.drop_duplicates()

In [20]:
INEGI_UNIQUEMUN_2020.shape

(2469, 7)

In [21]:
INEGI_UNIQUEMUN_2020.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

In [22]:
INEGI_UNIQUEMUN_2020.drop_duplicates(subset='KEY_inegi_municipio', keep='first', inplace=True)


# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2020

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2020, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_20 y el dataset de INEGI correspondiente.

In [23]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [24]:
diccionario_MUN_20 = fuzzy_merge_benef2019_2022(Municipios_20, INEGI_UNIQUEMUN_2020, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_20.shape

(182, 14)

In [25]:
diccionario_MUN_20.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)
diccionario_MUN_20.drop_duplicates(subset='KEY_benef_mun', keep='first', inplace=True)

In [26]:
diccionario_MUN_20.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_20.csv', index=False)

In [27]:
diccionario_MUN_20_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_20_simple.csv')

In [28]:
diccionario_MUN_21_simple = diccionario_MUN_20_simple.drop_duplicates(subset='KEY_inegi_municipio', keep='first')

In [29]:
diccionario_MUN_21_simple = diccionario_MUN_20_simple.drop_duplicates(subset='KEY_inegi_municipio', keep='first')

In [30]:
diccionario_MUN_20_simple.shape

(183, 2)

In [31]:
diccionario_MUN_20_simple.columns

Index(['KEY_benef_mun', 'KEY_inegi_municipio'], dtype='object')

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [32]:
beneficiarios_20.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [33]:
beneficiarios_20.shape

(369037, 16)

In [34]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_20['ESTADO_Clean'] = beneficiarios_20['ENTIDAD'].apply(clean_text)
beneficiarios_20['MUNICIPIO_Clean'] = beneficiarios_20['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_20['Estado-mun-KEY'] = beneficiarios_20['ESTADO_Clean'].astype(str) + '-' + beneficiarios_20[
    'MUNICIPIO_Clean'].astype(str)

In [35]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_20, diccionario_MUN_20_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left')

In [36]:
listado_beneficiarios_parte_I.shape

(369037, 21)

In [37]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [38]:
listado_beneficiarios_parte_I.isna().sum()

BENEFICIARIO           0
ZONA                   0
ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
ESTRATIFICACIÓN        0
PROGRAMA               0
COMPONENTE             0
SUBCOMPONENTE          0
PRODUCTO               0
FECHA                  0
MONTO FEDERAL          0
APOYO                  0
ACTIVIDAD              0
ESLABÓN                0
CICLO AGRÍCOLA         0
ESTADO_Clean           0
MUNICIPIO_Clean        0
Estado-mun-KEY         0
KEY_benef_mun          0
KEY_inegi_municipio    0
dtype: int64

In [39]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2020, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [40]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,ESPINOZA ZAMORA SILVANO,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
1,CASTRO IGNACIO ESTEBAN,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
2,ESPINOZA ESPINOZA AMELIA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
3,DIRCIO CALIXTO ANUNCIACION,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
4,CALIXTRO LOPEZ LAURA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369032,RAMIREZ FLORES GUILLERMINA,CENTRO PAÍS,PUEBLA,ACAJETE,LA MAGDALENA TETELA MORELOS,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acajete,puebla-acajete,puebla-acajete,puebla-acajete,21,Puebla,1,Acajete,puebla,acajete
369033,MARTINEZ HERNANDEZ JOSE JUAN,CENTRO PAÍS,PUEBLA,SAN SALVADOR EL SECO,SAN SALVADOR EL SECO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,san salvador el seco,puebla-san salvador el seco,puebla-san salvador el seco,puebla-san salvador el seco,21,Puebla,142,San Salvador el Seco,puebla,san salvador el seco
369034,VARILLAS MORA JUANA,CENTRO PAÍS,PUEBLA,TLACOTEPEC DE BENITO JUÁREZ,TLACOTEPEC DE BENITO JUÁREZ,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,21,Puebla,177,Tlacotepec de Benito Juárez,puebla,tlacotepec de benito juarez
369035,JIMENEZ AGUILAR FLORENTINO,CENTRO PAÍS,GUERRERO,JUCHITÁN,JUCHITÁN,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,juchitan,guerrero-juchitan,guerrero-juchitan,guerrero-juchitan,12,Guerrero,80,Juchitán,guerrero,juchitan


In [41]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi','Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [42]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [43]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2020.csv', index=False)

In [44]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

In [45]:
listado_beneficiarios_parte_II.shape

(369037, 14)

# 3. Join por entidad (Divide y Vencerás)

In [46]:
listado_beneficiarios_2020_match = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2020_match.csv')

In [47]:
listado_beneficiarios_2020_match.columns

Index(['Unnamed: 0', 'BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO',
       'LOCALIDAD', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA'],
      dtype='object')

In [48]:
# Obtenemos las localidades únicas en el dataset.
listado_beneficiarios_2020_match = listado_beneficiarios_2020_match.drop(columns=['Unnamed: 0'])

In [49]:
listado_beneficiarios_2020_match['ENTIDAD_c_benef'] = listado_beneficiarios_2020_match['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2020_match['MUNICIPIO_c_benef'] = listado_beneficiarios_2020_match['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2020_match['LOCALIDAD_c_benef'] = listado_beneficiarios_2020_match['LOCALIDAD'].apply(clean_text)

In [50]:
listado_beneficiarios_2020_match['KEY_benef_loc'] = listado_beneficiarios_2020_match['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2020_match['LOCALIDAD_c_benef'].astype(str)

In [51]:
listado_beneficiarios_2020_match.shape

(369091, 13)

In [52]:
listado_beneficiarios_2020_match = listado_beneficiarios_2020_match.drop_duplicates(subset=['KEY_benef_loc'])

In [53]:
listado_beneficiarios_2020_match.shape

(5404, 13)

In [54]:
listado_beneficiarios_2020_match.ENTIDAD.unique()

array(['GUERRERO', 'MORELOS', 'TLAXCALA', 'PUEBLA', 'Guerrero', 'Puebla'],
      dtype=object)

### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [55]:
entities = ['GUERRERO', 'MORELOS', 'TLAXCALA', 'PUEBLA', 'Guerrero', 'Puebla']

# Dictionary to hold DataFrames
dataframes_dict = {}

for entity in entities:
    variable_name = f"Localidades_20_{entity.upper().replace(' ', '_')}"
    dataframe = listado_beneficiarios_2020_match[listado_beneficiarios_2020_match["ENTIDAD"] == entity]
    
    # Check if the DataFrame for this entity already exists in the dictionary
    if variable_name in dataframes_dict:
        # If it exists, concatenate the new DataFrame with the existing one
        dataframes_dict[variable_name] = pd.concat([dataframes_dict[variable_name], dataframe])
    else:
        # If it does not exist, create a new entry in the dictionary
        dataframes_dict[variable_name] = dataframe

# Define each DataFrame as a separate variable
for name, df in dataframes_dict.items():
    globals()[name] = df
    print(f"{name} defined with shape: {df.shape}")

Localidades_20_GUERRERO defined with shape: (4554, 13)
Localidades_20_MORELOS defined with shape: (270, 13)
Localidades_20_TLAXCALA defined with shape: (290, 13)
Localidades_20_PUEBLA defined with shape: (290, 13)


In [56]:
Localidades_20_Guerrero = listado_beneficiarios_2020_match[listado_beneficiarios_2020_match['ENTIDAD'] == 'Guerrero']

In [57]:
listado_beneficiarios_2020_match.shape

(5404, 13)

### 3.2.2 Diccionarios de Inegi por entidad.

In [58]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2020'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


INEGI_UNIQUELOC_2020_puebla: 7726 filas, 11 columnas
INEGI_UNIQUELOC_2020_guerrero: 9220 filas, 11 columnas
INEGI_UNIQUELOC_2020_morelos: 1904 filas, 11 columnas
INEGI_UNIQUELOC_2020_tlaxcala: 1689 filas, 11 columnas


In [59]:
INEGI_UNIQUELOC_2020_guerrero.shape

(9220, 11)

In [60]:
Localidades_20_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_20_GUERRERO, INEGI_UNIQUELOC_2020_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_20_GUERRERO_parte_I.shape

Localidades_20_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_20_PUEBLA, INEGI_UNIQUELOC_2020_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_20_PUEBLA_parte_I.shape

Localidades_20_MORELOS_parte_I = fuzzy_merge_benef2019_2022(Localidades_20_MORELOS, INEGI_UNIQUELOC_2020_morelos, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_20_MORELOS_parte_I.shape

Localidades_20_TLAXCALA_parte_I = fuzzy_merge_benef2019_2022(Localidades_20_TLAXCALA, INEGI_UNIQUELOC_2020_tlaxcala, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_20_TLAXCALA_parte_I.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26870/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26870/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26870/734782849.py:18: SettingWithCopyWarnin

(290, 26)

In [61]:
# Concatenate datasets vertically
diccionario_LOC_20 = pd.concat([
    Localidades_20_GUERRERO_parte_I,
    Localidades_20_PUEBLA_parte_I,
    Localidades_20_MORELOS_parte_I,
    Localidades_20_TLAXCALA_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_20.shape

(5404, 26)

In [62]:
diccionario_LOC_20.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'ENTIDAD_c_benef',
       'MUNICIPIO_c_benef', 'LOCALIDAD_c_benef', 'best_match', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'Localidad_c_inegi'], inplace=True)

In [63]:
diccionario_LOC_20.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_20.csv', index=False)

In [64]:
diccionario_LOC_20.isna().sum()

KEY_benef_loc          0
match_score            0
KEY_inegi_localidad    0
dtype: int64

# 4. Diccionarios de localidades y merge respectivos.

Empezamos leyendo el diccionario_LOC_20_simple, el cual solo tiene key_benef_mun y key_inegi_municipio manipulada.

In [65]:
import pandas as pd

# Intentar leer el archivo CSV con diferentes configuraciones de lectura
try:
    diccionario_LOC_20_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_20_simple.csv', encoding='cp1252', 
                          delimiter=';')
    #diccionario_LOC_21_simple.drop(columns=['ValidaciÃ³n', 'Propuesta'], inplace=True)
    print(diccionario_LOC_20_simple.head())
except pd.errors.ParserError as e:
    print("Error al leer el archivo CSV:", e)




                                    ï»¿KEY_benef_loc  match_score  \
0  teloloapan-cuadrilla del encanto (cerro del ba...           86   
1  arcelia-reforma de lazaro cardenas (el aterriz...           86   
2                           arcelia-colonia progreso           86   
3               arcelia-colonia emperador cuauhtemoc           86   
4  pungarabato-seminario diocesano la anunciacion...           86   

                           KEY_inegi_localidad  
0                        teloloapan-teloloapan  
1                           arcelia-la reforma  
2  acapulco de juarez-colonia acapulco dos mil  
3                              arcelia-arcelia  
4                pungarabato-ciudad altamirano  


In [66]:
diccionario_LOC_20_simple.columns

Index(['ï»¿KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [67]:
diccionario_LOC_20_simple.rename(columns={'ï»¿KEY_benef_loc': 'KEY_benef_loc'}, inplace=True)

In [68]:
diccionario_LOC_20_simple.columns

Index(['KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [69]:
diccionario_LOC_20_simple

,KEY_benef_loc,match_score,KEY_inegi_localidad
0,teloloapan-cuadrilla del encanto (cerro del ba...,86,teloloapan-teloloapan
1,arcelia-reforma de lazaro cardenas (el aterriz...,86,arcelia-la reforma
2,arcelia-colonia progreso,86,acapulco de juarez-colonia acapulco dos mil
3,arcelia-colonia emperador cuauhtemoc,86,arcelia-arcelia
4,pungarabato-seminario diocesano la anunciacion...,86,pungarabato-ciudad altamirano
...,...,...,...
5399,contla de juan cuamatzi-san jose aztatla,100,contla de juan cuamatzi-san jose aztatla
5400,huamantla-el durazno,100,huamantla-el durazno
5401,atltzayanca-la garita,100,atltzayanca-la garita
5402,teolocholco-el carmen aztama,100,teolocholco-el carmen aztama


In [70]:
listado_beneficiarios_parte_II['MUNICIPIO_Clean'] = listado_beneficiarios_parte_II['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_parte_II['LOCALIDAD_Clean'] = listado_beneficiarios_parte_II['LOCALIDAD'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios_parte_II['Municipio-loc-KEY'] = listado_beneficiarios_parte_II['MUNICIPIO_Clean'].astype(str) + '-' + listado_beneficiarios_parte_II[
    'LOCALIDAD_Clean'].astype(str)

Primer merge entre listado_beneficiarios_parte_II, y diccionario_LOC_22_simple. En este merge cogemos el dataset con los municipios corregidos previamente y ejecutamos el merge de manera que nos traemos las key_inegi_localidad corregidas.

In [71]:
listado_beneficiarios_parte_II.shape

(369037, 17)

In [72]:
INEGI_UNIQUELOC_2020 = dataset_inegi_2020.drop(columns=['Entidad_c_inegi', 'Municipio_c_inegi',
       'Localidad_c_inegi', 'POB_TOTAL'])
INEGI_UNIQUELOC_2020 = INEGI_UNIQUELOC_2020.drop_duplicates(subset='KEY_inegi_localidad', keep='first')

In [73]:
INEGI_UNIQUELOC_2020.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'CVE_LOC',
       'Localidad_inegi', 'KEY_inegi_municipio', 'KEY_inegi_localidad'],
      dtype='object')

In [74]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop_duplicates()

In [75]:
listado_beneficiarios_parte_II['Municipio-loc-KEY'] = listado_beneficiarios_parte_II['Municipio-loc-KEY'].str.strip().str.lower()
diccionario_LOC_20_simple['KEY_benef_loc'] = diccionario_LOC_20_simple['KEY_benef_loc'].str.strip().str.lower()

# Asegurar que las keys son del mismo tipo
listado_beneficiarios_parte_II['Municipio-loc-KEY'] = listado_beneficiarios_parte_II['Municipio-loc-KEY'].astype(str)
diccionario_LOC_20_simple['KEY_benef_loc'] = diccionario_LOC_20_simple['KEY_benef_loc'].astype(str)

In [76]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I_localidades = pd.merge(listado_beneficiarios_parte_II, diccionario_LOC_20_simple, left_on="Municipio-loc-KEY",
                                         right_on="KEY_benef_loc", how='left', suffixes=('_benef', '_inegi'))

In [77]:
listado_beneficiarios_parte_I_localidades.shape

(368920, 20)

In [78]:
# Marcar todos los duplicados en las columnas 'columna1' y 'columna2' como True
duplicados = listado_beneficiarios_parte_I_localidades.duplicated(subset=['FECHA', 'BENEFICIARIO'], keep=False)

# Filtrar el DataFrame original para mostrar solo los duplicados
df_duplicados = listado_beneficiarios_parte_I_localidades[duplicados]

In [79]:
df_duplicados

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,MUNICIPIO_Clean,LOCALIDAD_Clean,Municipio-loc-KEY,KEY_benef_loc,match_score,KEY_inegi_localidad
1001,INFANTE HERNANDEZ RAMON,CENTRO PAÍS,GUERRERO,ZIHUATANEJO DE AZUETA,ZIHUATANEJO,Cultivo de maíz grano,2020-05-26,6163.44,PV2020,guerrero-zihuatanejo de azueta,12,Guerrero,38,Zihuatanejo de Azueta,zihuatanejo de azueta,zihuatanejo,zihuatanejo de azueta-zihuatanejo,zihuatanejo de azueta-zihuatanejo,100,zihuatanejo de azueta-zihuatanejo
1002,SANCHEZ SALAZAR JUAN,CENTRO PAÍS,GUERRERO,QUECHULTENANGO,JALAPA,Cultivo de maíz grano,2020-06-04,6163.44,PV2020,guerrero-quechultenango,12,Guerrero,51,Quechultenango,quechultenango,jalapa,quechultenango-jalapa,quechultenango-jalapa,100,quechultenango-jalapa
1251,MIER RAMIREZ NOLBERTO,CENTRO PAÍS,GUERRERO,JUCHITÁN,AGUA ZARCA,Cultivo de maíz grano,2020-05-22,3081.72,PV2020,guerrero-juchitan,12,Guerrero,80,Juchitán,juchitan,agua zarca,juchitan-agua zarca,juchitan-agua zarca,100,juchitan-agua zarca
1451,SANTOS MORALES JOSE,CENTRO PAÍS,GUERRERO,OMETEPEC,LA LADRILLERA,Cultivo de maíz grano,2020-06-22,3081.72,PV2020,guerrero-ometepec,12,Guerrero,46,Ometepec,ometepec,la ladrillera,ometepec-la ladrillera,ometepec-la ladrillera,100,ometepec-la ladrillera
2574,GALLEGOS MORALES APOLINAR,CENTRO PAÍS,GUERRERO,CUAUTEPEC,HUAMUCHITITÁN (EL HUAMUCHITO),Cultivo de maíz grano,2020-05-25,3081.72,PV2020,guerrero-cuautepec,12,Guerrero,25,Cuautepec,cuautepec,huamuchititan (el huamuchito),cuautepec-huamuchititan (el huamuchito),cuautepec-huamuchititan (el huamuchito),100,cuautepec-huamuchititan (el huamuchito)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356242,DOLORES ROSALES JUAN,CENTRO PAÍS,PUEBLA,TLACOTEPEC DE BENITO JUÁREZ,TLACOTEPEC DE BENITO JUÁREZ,Cultivo de maíz grano,2021-02-22,5784.84,PV2020,puebla-tlacotepec de benito juarez,21,Puebla,177,Tlacotepec de Benito Juárez,tlacotepec de benito juarez,tlacotepec de benito juarez,tlacotepec de benito juarez-tlacotepec de beni...,tlacotepec de benito juarez-tlacotepec de beni...,100,tlacotepec de benito juarez-tlacotepec de beni...
358774,GONZALEZ CRUZ ORLANDO,CENTRO PAÍS,TLAXCALA,LÁZARO CÁRDENAS,LÁZARO CÁRDENAS,Cultivo de maíz grano,2021-02-26,5577.06,PV2020,tlaxcala-lazaro cardenas,29,Tlaxcala,47,Lázaro Cárdenas,lazaro cardenas,lazaro cardenas,lazaro cardenas-lazaro cardenas,lazaro cardenas-lazaro cardenas,100,lazaro cardenas-lazaro cardenas
358975,GONZALEZ CRUZ ORLANDO,CENTRO PAÍS,TLAXCALA,LÁZARO CÁRDENAS,LÁZARO CÁRDENAS,Cultivo de maíz grano,2021-02-26,5732.92,PV2020,tlaxcala-lazaro cardenas,29,Tlaxcala,47,Lázaro Cárdenas,lazaro cardenas,lazaro cardenas,lazaro cardenas-lazaro cardenas,lazaro cardenas-lazaro cardenas,100,lazaro cardenas-lazaro cardenas
364568,FLORES ACALCO BENJAMIN,CENTRO PAÍS,PUEBLA,ACATZINGO,ACTIPAN DE MORELOS (SANTA MARÍA ACTIPAN),Cultivo de maíz grano,2021-03-09,2892.42,PV2020,puebla-acatzingo,21,Puebla,4,Acatzingo,acatzingo,actipan de morelos (santa maria actipan),acatzingo-actipan de morelos (santa maria acti...,acatzingo-actipan de morelos (santa maria acti...,100,acatzingo-actipan de morelos (santa maria acti...


In [80]:
listado_beneficiarios_parte_I_localidades.drop_duplicates(keep='first', inplace=True)

In [81]:
listado_beneficiarios_parte_I_localidades.shape

(368920, 20)

In [82]:
listado_beneficiarios_parte_I_localidades.isna().sum()

BENEFICIARIO           0
ZONA                   0
ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
PRODUCTO               0
FECHA                  0
MONTO FEDERAL          0
CICLO AGRÍCOLA         0
KEY_benef_mun          0
CVE_ENT                0
Entidad_inegi          0
CVE_MUN                0
Municipio_inegi        0
MUNICIPIO_Clean        0
LOCALIDAD_Clean        0
Municipio-loc-KEY      0
KEY_benef_loc          0
match_score            0
KEY_inegi_localidad    0
dtype: int64

In [83]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_I_localidades[listado_beneficiarios_parte_I_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [84]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,MUNICIPIO_Clean,LOCALIDAD_Clean,Municipio-loc-KEY,KEY_benef_loc,match_score,KEY_inegi_localidad


In [85]:
print(listado_beneficiarios_parte_I_localidades['KEY_inegi_localidad'].duplicated().sum())
print(INEGI_UNIQUELOC_2020['KEY_inegi_localidad'].duplicated().sum())

363611
0


In [86]:
# TODO: REVISAR ESTA PARTE (ARTURO)
listado_beneficiarios_parte_II_localidades = pd.merge(listado_beneficiarios_parte_I_localidades, INEGI_UNIQUELOC_2020,
                                          left_on="KEY_inegi_localidad",
                                          right_on="KEY_inegi_localidad", how='left', suffixes=('_benef', '_inegi'))

In [87]:
listado_beneficiarios_parte_II_localidades.isna().sum()

BENEFICIARIO             0
ZONA                     0
ENTIDAD                  0
MUNICIPIO                0
LOCALIDAD                0
PRODUCTO                 0
FECHA                    0
MONTO FEDERAL            0
CICLO AGRÍCOLA           0
KEY_benef_mun            0
CVE_ENT_benef            0
Entidad_inegi_benef      0
CVE_MUN_benef            0
Municipio_inegi_benef    0
MUNICIPIO_Clean          0
LOCALIDAD_Clean          0
Municipio-loc-KEY        0
KEY_benef_loc            0
match_score              0
KEY_inegi_localidad      0
CVE_ENT_inegi            0
Entidad_inegi_inegi      0
CVE_MUN_inegi            0
Municipio_inegi_inegi    0
CVE_LOC                  0
Localidad_inegi          0
KEY_inegi_municipio      0
dtype: int64

In [88]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II_localidades[listado_beneficiarios_parte_II_localidades.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [89]:
rows_with_nan.tail(50)

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,...,KEY_benef_loc,match_score,KEY_inegi_localidad,CVE_ENT_inegi,Entidad_inegi_inegi,CVE_MUN_inegi,Municipio_inegi_inegi,CVE_LOC,Localidad_inegi,KEY_inegi_municipio


In [90]:
listado_beneficiarios_parte_II_localidades.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'MUNICIPIO_Clean', 'LOCALIDAD_Clean',
       'Municipio-loc-KEY', 'KEY_benef_loc', 'match_score',
       'KEY_inegi_localidad', 'CVE_ENT_inegi', 'Entidad_inegi_inegi',
       'CVE_MUN_inegi', 'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi',
       'KEY_inegi_municipio'],
      dtype='object')

In [91]:
listado_beneficiarios_parte_II_localidades.drop(columns=['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'MUNICIPIO_Clean', 'LOCALIDAD_Clean', 'Municipio-loc-KEY', 'match_score', 'KEY_inegi_localidad',
       'KEY_inegi_municipio', 'KEY_benef_mun', 'CVE_ENT_benef',
       'Entidad_inegi_benef', 'CVE_MUN_benef', 'Municipio_inegi_benef',
       'KEY_benef_loc'], inplace=True)

In [92]:
listado_beneficiarios_parte_II_localidades.columns

Index(['BENEFICIARIO', 'ZONA', 'PRODUCTO', 'FECHA', 'MONTO FEDERAL',
       'CICLO AGRÍCOLA', 'CVE_ENT_inegi', 'Entidad_inegi_inegi',
       'CVE_MUN_inegi', 'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi'],
      dtype='object')

In [93]:
listado_beneficiarios_parte_II_localidades.rename(columns={'BENEFICIARIO': 'Nombre del beneficiario',
                                                           'ZONA': 'Zona del país',
                                                           'CVE_ENT_inegi': 'Clave de entidad',
                                                            'Entidad_inegi_inegi': 'Entidad federativa',
                                                            'CVE_MUN_inegi': 'Clave de municipio',
                                                            'Municipio_inegi_inegi': 'Municipio',                                                           
                                                            'CVE_LOC': 'Clave de localidad',
                                                            'Localidad_inegi': 'Localidad',
                                                            'PRODUCTO': 'Producto cultivado',
                                                            'FECHA': 'Fecha de entrega del beneficio',
                                                            'MONTO FEDERAL': 'Monto entregado',
                                                            'CICLO AGRÍCOLA': 'Ciclo agrícola'}, inplace=True)

In [94]:
listado_beneficiarios_parte_II_localidades.to_csv('../../data/listados_completos/listado_beneficiarios_2020_localidades.csv', index=False)